In [49]:
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [17]:
df = pd.read_csv("rawdata.csv")


features :
- brawler 1-6
- map

In [18]:
firstTeams = []
secondTeams = []

for index, row in df.iterrows():
    team1 = []
    team2 = []
    team1.append(row["Brawler1Name"])
    team1.append(row["Brawler2Name"])
    team1.append(row["Brawler3Name"])
    team2.append(row["Brawler4Name"])
    team2.append(row["Brawler5Name"])
    team2.append(row["Brawler6Name"])
    team1 = sorted(team1)
    team2 = sorted(team2)
    firstTeams.append(team1)
    secondTeams.append(team2)

combinedFirstTeams = []
combinedSecondTeams = []

for i in range(len(firstTeams)):
    team1 = ""
    team1 = team1 + firstTeams[i][0]
    team1 = team1 + firstTeams[i][1]
    team1 = team1 + firstTeams[i][2]
    combinedFirstTeams.append(team1)
    team2 = ""
    team2 = team2 + secondTeams[i][0]
    team2 = team2 + secondTeams[i][1]
    team2 = team2 + secondTeams[i][2]
    combinedSecondTeams.append(team2)

In [23]:
dfFinal = pd.DataFrame(columns=["Team1", "Team2", "map"])
dfFinal["Team1"] = combinedFirstTeams
dfFinal["Team2"] = combinedSecondTeams
dfFinal["map"] =df["map"]
dfFinal["WinningTeam"] = df["WinningTeam"]

encoding = LabelEncoder()
encoding.fit(combinedFirstTeams + combinedSecondTeams)
dfFinal["Team1"] = encoding.transform(dfFinal["Team1"].values)
dfFinal["Team2"] = encoding.transform(dfFinal["Team2"].values)

# encode maps
encoding = LabelEncoder()
encoding.fit(dfFinal["map"].values)
dfFinal["map"] = encoding.transform(dfFinal["map"].values)

dfFinal.head(20)

,Team1,Team2,map,WinningTeam
0,172,73,0,1
1,172,949,0,1
2,172,456,0,1
3,172,865,0,1
4,172,459,0,1
5,172,497,0,1
6,225,172,0,2
7,963,172,0,2
8,169,172,0,2
9,976,175,11,1


In [56]:
x_features = []
winners = []

dfFinal = dfFinal[dfFinal["map"] == 0]

for index, row in dfFinal.iterrows():
    features = []
    #features.append(row["map"])
    features.append(row["Team1"])
    features.append(row["Team2"])

    x_features.append(features)
    winners.append(row["WinningTeam"])

In [39]:
tree1 = tree.DecisionTreeClassifier()
tree1 = tree1.fit(x_features, winners)

In [57]:
tree2 = RandomForestClassifier()
tree2 = tree2.fit(x_features, winners)

In [58]:
scores = cross_val_score(tree2, x_features, winners, scoring='accuracy')
np.mean(scores)

0.6113443830570902